## Extraction of gps from mdb's

In [1]:
import pyodbc,os,pandas as pd

mdb_directory = r"D:\_2.Programming\_6.test\mdb_processed" # input directory
o_gps = r"D:\_2.Programming\_6.test\GPS.xlsx" # output directory

#defining function to read mdb's
def read_mdb(x):
    mdb_files = []
    for root, dirs, files in os.walk(x):
        for file in files:
            if file.endswith(".mdb"):
                mdb_files.append(os.path.join(root, file))
    return mdb_files

#defining function to read tables in mdb
def read_tables_by_prefix(mdb_file, table_prefix):
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};Dbq={mdb_file};"
    connection = pyodbc.connect(conn_str)
    cursor = connection.cursor()

    tables = []
    for table_info in cursor.tables(tableType='TABLE'):
        table_name = table_info.table_name
        if table_name.startswith(table_prefix):
            tables.append(table_name)

    data = pd.DataFrame()
    for table in tables:
        query = f"SELECT * FROM [{table}]"
        table_data = pd.read_sql(query, connection)
        table_data['Survey_ID_1'] = os.path.splitext(os.path.basename(mdb_file))[0]
        data = pd.concat([data, table_data], ignore_index=True)

    connection.close()
    return data

#calling function to read mdb's
mdb_files = read_mdb(mdb_directory)

#pandas data frame for storing extracted gps and keycode
GPS_P = pd.DataFrame()

#extracting and merging gps and keycode from mdb's 
for mdb_file in mdb_files:
    gps = read_tables_by_prefix(mdb_file, "dt_GPS_Processed")
    GPS_P = pd.concat([GPS_P, gps], ignore_index=True)

#exporting gps and keycode from pandas data frame
GPS_P.to_excel(o_gps, index=False)

print(len(mdb_files))
print("Successfully completed!!")


6
Successfully completed!!
